In [24]:
import json
import re

In [30]:
with open("mer_final_emotion_caps2emotion.json", "r", encoding="utf-8") as f:
    content = f.read()

content=content.replace("'id'","\"id\"").replace("'emotion'","\"emotion\"").replace("'sample","\"sample")
lines = content.split("\n")
result = ''
for line in lines:
    modified_line = re.sub("\'","\"",line,count=1)
    result += modified_line + "\n"
result

with open("result.json", "w", encoding="utf-8") as f:
    f.write(result)

In [31]:
with open("result.json", "r", encoding="utf-8") as f:
    content = f.read()

lines = content.split("\n")
result = '{'
for line in lines:
    result += line + ",\n"
result+="}"
with open("result.json", "w", encoding="utf-8") as f:
    f.write(result)

In [33]:
with open("result.json", "r", encoding="utf-8") as f:
    data = json.load(f)

data["data"]

[{'id': 'sample_00000000', 'emotion': "['悲伤', '痛苦', '难受', '伤心', '激动']"},
 {'id': 'sample_00000007', 'emotion': "['烦躁', '不耐烦', '沮丧']"},
 {'id': 'sample_00000021', 'emotion': "['关心', '认真', '担忧', '忧伤', '信赖', '无奈']"},
 {'id': 'sample_00000033', 'emotion': "['悲伤', '痛苦', '沮丧', '内疚']"},
 {'id': 'sample_00000039', 'emotion': "['愤怒', '激动']"},
 {'id': 'sample_00000055', 'emotion': "['悲伤', '忧伤', '痛苦', '凄凉']"},
 {'id': 'sample_00000068', 'emotion': "['悲伤', '忧伤', '痛苦', '无奈']"},
 {'id': 'sample_00000070', 'emotion': "['悲伤', '伤心', '难过', '忧伤']"},
 {'id': 'sample_00000073', 'emotion': "['愤怒', '不满', '恼怒', '痛苦', '悲伤']"},
 {'id': 'sample_00000114',
  'emotion': "['悲伤', '痛苦', '苦闷', '心酸', '难过', '忐忑', '不安']"},
 {'id': 'sample_00000118', 'emotion': "['悲伤', '委屈', '难过', '激动', '情绪激动']"},
 {'id': 'sample_00000126', 'emotion': "['疑惑', '沮丧', '不理解', '感触']"},
 {'id': 'sample_00000153', 'emotion': "['疑问', '不安', '不确定']"},
 {'id': 'sample_00000166', 'emotion': "['平静', '舒缓']"},
 {'id': 'sample_00000174',
  'emotion': "['

In [1]:
import pandas as pd

In [10]:
df = pd.read_csv("final-openset-chinese.csv")
df[:1]

,name,EMER,Empty,Random,Valley,Otter,PandaGPT,Video-LLaMA,VideoChat2,Video-ChatGPT,VideoChat,SALMONN,mPLUG-Owl,Qwen-Audio,GPT-4V
0,sample_00000000,"['轻松', '愉快', '幽默', '自嘲']",[],['担忧'],"['专注', '工作', '专业']",['不确定'],['微笑'],['焦虑'],"['惊讶', '困惑', '迷茫']",['困惑'],"['自信', '满意', '参与', '与工作相关的目的或知识的混合', '专业', '专注']","['沮丧', '失望', '压力山大', '焦虑', '不足', '怀疑自己']","['不安全感', '紧张', '不确定性']",['困惑'],"['无助', '尴尬']"


In [27]:
df.iloc[:,1:].to_numpy()

array([["['轻松', '愉快', '幽默', '自嘲']", '[]', "['担忧']", ...,
        "['不安全感', '紧张', '不确定性']", "['困惑']", "['无助', '尴尬']"],
       ["['高兴', '期待']", '[]', "['担忧']", ...,
        "['娱乐', '困惑', '轻松', '玩笑', '友好']", "['开心']", "['积极']"],
       ["['疑虑', '担忧']", '[]', "['惊讶']", ..., "['焦虑', '不确定']", "['愿意']",
        "['平静', '专注']"],
       ...,
       ["['焦虑', '紧张']", '[]', "['生气']", ..., "['伤心', '不高兴']",
        "['惊讶', '震惊', '悲伤']", "['失望', '惊讶']"],
       ["['痛苦', '崩溃']", '[]', "['悲伤']", ..., "['注意', '沮丧', '不满', '紧张']",
        "['生气']", "['紧张', '冲突']"],
       ["['生气', '愤怒']", '[]', "['惊讶']", ..., "['困惑', '沮丧', '不确定', '不满']",
        "['中性']", "['紧张', '警觉', '困惑', '期待', '不确定']"]], dtype=object)

In [15]:
def get_accuracy(prediction_ids: set[str], true_ids: set[str]):
    return len(prediction_ids.intersection(true_ids)) / len(prediction_ids)

def get_recall(prediction_ids: set[str], true_ids: set[str]):
    return len(prediction_ids.intersection(true_ids)) / len(true_ids)

def get_score(prediction_ids: set[str], true_ids: set[str]):
    return (get_accuracy(prediction_ids, true_ids) + get_recall(prediction_ids, true_ids)) / 2

In [14]:
get_accuracy(set(["1", "2", "3"]), set(["4", "2", "3"]))

0.6666666666666666

In [3]:
import openai
import ast


client = openai.OpenAI(api_key="sk-SPmgatijZMoa4g4WE9Ea5c9d0a124d4dBb43418b88C0Ae69", base_url="https://api.xi-ai.cn/v1")

prompt = "请扮演情感领域的专家。我们将提供一组情绪。请将这些情绪分组，每组包含同义词或一致的情感术语。直接以Python列表形式输出结果。"


def get_emotion_ids(prediction_emotions: list[str], true_emotions: list[str]) -> tuple[set[int], set[int]]:

    all_emotions = set(prediction_emotions + true_emotions)

    completion = client.chat.completions.create(

        model="gpt-3.5-turbo", messages=[{"role": "user", "content": f"{prompt}\n{all_emotions}"}]
    )

    arrs = ast.literal_eval(completion.choices[0].message.content)
    print(arrs)
    prediction_ids = set()
    true_ids = set()
    for emotion in prediction_emotions:
        for i, arr in enumerate(arrs):
            if emotion in arr:
                prediction_ids.add(i)
    for emotion in true_emotions:
        for i, arr in enumerate(arrs):
            if emotion in arr:
                true_ids.add(i)
    return prediction_ids, true_ids

In [4]:
get_emotion_ids(["开心", "快乐", "忧伤"], ["愤怒", "生气"])

[['忧伤'], ['愤怒', '生气'], ['快乐', '开心']]


({0, 2}, {1})

In [29]:
t = ["2","s","c"]
print(f"{t}")

['2', 's', 'c']
